# Objective Definition:

- Present a strategic recommendation to Julia for an upcoming category review, supported by data analysis.
- Focus on understanding current purchasing trends and behaviors, specifically in customer segments and chip purchasing.



# Data Description

### transaction Information:


- **DATE**: The date of the transaction.
- **STORE_NBR**: Store number or identifier where the transaction occurred.
- **LYLTY_CARD_NBR**: Loyalty card number associated with the customer making the transaction.
- **TXN_ID**: Transaction ID, a unique identifier for each transaction.
- **PROD_NBR**: Product number or identifier for the item being purchased.
- **PROD_NAME**: Name of the product.
- **PROD_QTY**: Quantity of the product purchased in the transaction.
- **TOT_SALES**: Total sales amount for the transaction.

### Customer Information:
- **LYLTY_CARD_NBR**: Loyalty card number associated with the customer making the transaction.
- **LIFESTAGE**: The life stage of the customer, categorized as YOUNG SINGLES/COUPLES, YOUNG FAMILIES, OLDER SINGLES/COUPLES, MIDAGE SINGLES/COUPLES, etc.
- **PREMIUM_CUSTOMER**: The type of customer loyalty program the customer is subscribed to, categorized as Premium, Mainstream, or Budget.



# Import libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# We will start with transaction data

In [3]:
transaction = pd.read_excel('/content/QVI_transaction_data.xlsx')

In [4]:
transaction.head()

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES
0,43390,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.0
1,43599,1,1307,348,66,CCs Nacho Cheese 175g,3,6.3
2,43605,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.9
3,43329,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0
4,43330,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.8


In [5]:
transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264836 entries, 0 to 264835
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   DATE            264836 non-null  int64  
 1   STORE_NBR       264836 non-null  int64  
 2   LYLTY_CARD_NBR  264836 non-null  int64  
 3   TXN_ID          264836 non-null  int64  
 4   PROD_NBR        264836 non-null  int64  
 5   PROD_NAME       264836 non-null  object 
 6   PROD_QTY        264836 non-null  int64  
 7   TOT_SALES       264836 non-null  float64
dtypes: float64(1), int64(6), object(1)
memory usage: 16.2+ MB


- The "Date" column should be converted to a date type.
- The "PROD_NAME" column should be converted to a category.

In [6]:
transaction.isnull().sum()

DATE              0
STORE_NBR         0
LYLTY_CARD_NBR    0
TXN_ID            0
PROD_NBR          0
PROD_NAME         0
PROD_QTY          0
TOT_SALES         0
dtype: int64

In [7]:
transaction.duplicated().sum()

1

In [8]:
transaction.drop_duplicates(subset=['TXN_ID'], inplace=True)

# Data Preprocessing

In [9]:
# Change 'DATE' column to date type
transaction['DATE'] = pd.to_datetime(transaction['DATE'], origin='1899-12-30', unit='D')

In [10]:
transaction['PROD_NAME'] = transaction['PROD_NAME'].astype('category')

In [11]:
print(transaction['PROD_NAME'].unique())

['Natural Chip        Compny SeaSalt175g', 'CCs Nacho Cheese    175g', 'Smiths Crinkle Cut  Chips Chicken 170g', 'Smiths Chip Thinly  S/Cream&Onion 175g', 'Kettle Tortilla ChpsHny&Jlpno Chili 150g', ..., 'Red Rock Deli Chikn&Garlic Aioli 150g', 'RRD SR Slow Rst     Pork Belly 150g', 'RRD Pc Sea Salt     165g', 'Smith Crinkle Cut   Bolognese 150g', 'Doritos Salsa Mild  300g']
Length: 114
Categories (114, object): ['Burger Rings 220g', 'CCs Nacho Cheese    175g', 'CCs Original 175g',
                           'CCs Tasty Cheese    175g', ..., 'WW Supreme Cheese   Corn Chips 200g',
                           'Woolworths Cheese   Rings 190g', 'Woolworths Medium   Salsa 300g',
                           'Woolworths Mild     Salsa 300g']


In [12]:
# Extract 'size' from 'PROD_NAME'
transaction['size'] = transaction['PROD_NAME'].str.extract('(\d+)').astype('int')

In [13]:
# Extract 'brand' from 'PROD_NAME'
transaction['brand'] = transaction.PROD_NAME.str.split(' ', 1).str[0]

<ipython-input-13-f1ba30d23bea>:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  transaction['brand'] = transaction.PROD_NAME.str.split(' ', 1).str[0]


In [14]:
# Map 'brand' names
brand_mapper = {'red':'RRD','ww':'WOOLWORTHS','ncc':'NATURAL','snbts':'SUNBITES','infzns':'INFUZIONS','smith':'SMITHS','dorito':'DORITOS','grain':'GRNWVES'}
transaction['brand'] = transaction['brand'].str.lower().replace(brand_mapper).str.upper()

In [15]:
# Map and standardize 'brand' names again after replacement
transaction['PROD_NAME'] = transaction['PROD_NAME'].replace(brand_mapper)

In [16]:
# Clean 'PROD_NAME'
transaction['PROD_NAME'] = (
    transaction['PROD_NAME']
    .str.lower()
    .str.replace('\d+g', '')
    .str.replace('\s+', ' ')
    .str.strip()
    .str.title()
)

<ipython-input-16-df8b4c4d2e40>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace('\d+g', '')
<ipython-input-16-df8b4c4d2e40>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace('\s+', ' ')


In [17]:
# Replace specific substrings in 'PROD_NAME'
replacement_mapper = {'Ww': 'Woolworths', 'Ncc': 'Natural Chip Compny', 'Snbts': 'Sunbites', 'Infzns': 'Infuzions', 'Smith': 'Smiths', 'Dorito': 'Doritos', 'Grain': 'Grnwves', 'Rrd': 'Red Rock Deli'}
transaction['PROD_NAME'] = transaction['PROD_NAME'].replace(replacement_mapper, regex=True)

In [18]:
# Manager does not interact with salsa products.
transaction = transaction[~transaction['PROD_NAME'].str.contains('Salsa')]

In [19]:
print(transaction['PROD_NAME'].unique())

['Natural Chip Compny Seasalt' 'Ccs Nacho Cheese'
 'Smithss Crinkle Cut Chips Chicken' 'Smithss Chip Thinly S/Cream&Onion'
 'Kettle Tortilla Chpshny&Jlpno Chili'
 'Smithss Crinkle Chips Salt & Vinegar' 'Grnwves Waves Sweet Chilli'
 'Doritoss Corn Chip Mexican Jalapeno' 'Grnwves Waves Sour Cream&Chives'
 'Kettle Sensations Siracha Lime' 'Twisties Cheese'
 'Woolworths Crinkle Cut Chicken' 'Thins Chips Light& Tangy'
 'Ccs Original' 'Burger Rings'
 'Natural Chip Compny Sour Cream & Garden Chives'
 'Doritoss Corn Chip Southern Chicken' 'Cheezels Cheese Box'
 'Smithss Crinkle Original' 'Infuzions Crn Crnchers Tangy Gcamole'
 'Kettle Sea Salt And Vinegar' 'Smithss Chip Thinly Cut Original'
 'Kettle Original' 'Red Rock Deli Thai Chilli&Lime'
 'Pringles Sthrn Friedchicken' 'Pringles Sweet&Spcy Bbq'
 'Thins Chips Originl Saltd' 'Red Rock Deli Sp Salt & Truffle'
 'Smithss Thinly Swt Chli&S/Cream' 'Kettle Chilli' 'Doritoss Mexicana'
 'Smithss Crinkle Cut French Oniondip' 'Natural Chipco Hony Soy C

In [20]:
transaction.head()

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,size,brand
0,2018-10-17,1,1000,1,5,Natural Chip Compny Seasalt,2,6.0,175,NATURAL
1,2019-05-14,1,1307,348,66,Ccs Nacho Cheese,3,6.3,175,CCS
2,2019-05-20,1,1343,383,61,Smithss Crinkle Cut Chips Chicken,2,2.9,170,SMITHS
3,2018-08-17,2,2373,974,69,Smithss Chip Thinly S/Cream&Onion,5,15.0,175,SMITHS
4,2018-08-18,2,2426,1038,108,Kettle Tortilla Chpshny&Jlpno Chili,3,13.8,150,KETTLE


In [21]:
transaction.drop(['LYLTY_CARD_NBR', 'TXN_ID','STORE_NBR','PROD_NBR'], axis=1).describe()

,PROD_QTY,TOT_SALES,size
count,245143.000000,245143.000000,245143.000000
mean,1.907764,7.321015,175.591471
std,0.661684,3.081952,59.442959
min,1.000000,1.700000,70.000000
25%,2.000000,5.800000,150.000000
50%,2.000000,7.400000,170.000000
75%,2.000000,8.800000,175.000000
max,200.000000,650.000000,380.000000


In [22]:
print(transaction.PROD_QTY.unique())

[  2   3   5   1   4 200]


In [24]:
transaction.loc[transaction['PROD_QTY'] == 200]

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,size,brand
69762,2018-08-19,226,226000,226201,4,Doritos Corn Chp Supreme,200,650.0,380,DORITOS
69763,2019-05-20,226,226000,226210,4,Doritos Corn Chp Supreme,200,650.0,380,DORITOS


**Unexpected PROD_QTY in the transaction (200)😮**

# Let's continue with customer behavior data

In [25]:
behaviour = pd.read_csv('/content/QVI_purchase_behaviour.csv')

In [29]:
behaviour.head()

,LYLTY_CARD_NBR,LIFESTAGE,PREMIUM_CUSTOMER
0,1000,YOUNG SINGLES/COUPLES,Premium
1,1002,YOUNG SINGLES/COUPLES,Mainstream
2,1003,YOUNG FAMILIES,Budget
3,1004,OLDER SINGLES/COUPLES,Mainstream
4,1005,MIDAGE SINGLES/COUPLES,Mainstream


In [28]:
behaviour.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72637 entries, 0 to 72636
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   LYLTY_CARD_NBR    72637 non-null  int64 
 1   LIFESTAGE         72637 non-null  object
 2   PREMIUM_CUSTOMER  72637 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.7+ MB


In [30]:
behaviour.duplicated().any()

False

In [31]:
print(behaviour.LIFESTAGE.unique())

['YOUNG SINGLES/COUPLES' 'YOUNG FAMILIES' 'OLDER SINGLES/COUPLES'
 'MIDAGE SINGLES/COUPLES' 'NEW FAMILIES' 'OLDER FAMILIES' 'RETIREES']


In [32]:
print(behaviour.PREMIUM_CUSTOMER.unique())

['Premium' 'Mainstream' 'Budget']


In [33]:
behaviour['LIFESTAGE'] = behaviour['LIFESTAGE'].astype('category')
behaviour['PREMIUM_CUSTOMER'] = behaviour['PREMIUM_CUSTOMER'].astype('category')

# Combine data into one data frame

In [36]:
df = transaction.join(behaviour.set_index('LYLTY_CARD_NBR'), on = 'LYLTY_CARD_NBR')

In [37]:
df.head()

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,size,brand,LIFESTAGE,PREMIUM_CUSTOMER
0,2018-10-17,1,1000,1,5,Natural Chip Compny Seasalt,2,6.0,175,NATURAL,YOUNG SINGLES/COUPLES,Premium
1,2019-05-14,1,1307,348,66,Ccs Nacho Cheese,3,6.3,175,CCS,MIDAGE SINGLES/COUPLES,Budget
2,2019-05-20,1,1343,383,61,Smithss Crinkle Cut Chips Chicken,2,2.9,170,SMITHS,MIDAGE SINGLES/COUPLES,Budget
3,2018-08-17,2,2373,974,69,Smithss Chip Thinly S/Cream&Onion,5,15.0,175,SMITHS,MIDAGE SINGLES/COUPLES,Budget
4,2018-08-18,2,2426,1038,108,Kettle Tortilla Chpshny&Jlpno Chili,3,13.8,150,KETTLE,MIDAGE SINGLES/COUPLES,Budget
